# 01 Trendtopics with Tweepy
by Anderson Tavares

follow me on social media for more content

linkedin.com/in/anderson-tavares/

instagram.com/

github.com/ands-tavares/

In [ ]:
#Getting twitter authentification
#Credentials on https://developer.twitter.com/en/
import tweepy

consumer_key=''
consumer_secret=''
access_token=''
access_token_secret=''

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [ ]:
#Getting places with trends available
import numpy as np

places=api.trends_available()
places[np.random.randint(len(places))]

In [ ]:
##Creating a dataframe for places data
import pandas as pd

places_df = pd.DataFrame(np.array([place['name'] for place in places]), columns=['name']) 
places_df['placeTypeName']=np.array([place['placeType']['name'] for place in places])
places_df['country']=np.array([place['country'] for place in places])
places_df['woeid']=np.array([place['woeid'] for place in places])

places_df

In [ ]:
#Getting trends from a specific place
place=places_df.sample()
print(place)
print('\nTrendings from %s:\n' % place.name.values[0])
api.trends_place(place.woeid.values[0])

In [ ]:
#Getting dataframes from all places and saving in a dataframe
import time

worldtrends_df=pd.DataFrame(columns=['name','volume','place','placeTypeName','country'])
total=len(places_df)

start = time.time()

for index, row in places_df.iterrows():
  place=row['name']
  country=row['country']
  woeid=row['woeid']
  placeTypeName=row['placeTypeName']
  
  #Twitter api code. In case of an error, wait 15 minutes
  succeed=False
  while not succeed:
    try:
      trends=api.trends_place(woeid)
    except:
      print('Complete percentage: %.1f' % (index/total))
      print('Rate limit exceeded, wait 15 minutes.')
      time.sleep(5*60)
      print('Rate limit exceeded, wait 10 minutes.')
      time.sleep(5*60)
      print('Rate limit exceeded, wait 05 minutes.')
      time.sleep(5*60)
      print('Regular requests')
    else:
      succeed=True
  
  for trend in trends[0]['trends']:
    if trend['tweet_volume'] is not None:
      name=trend['name']
      volume=trend['tweet_volume']
      worldtrends_df=worldtrends_df.append({'name':name,'volume':volume,'place':place,'placeTypeName':placeTypeName,'country':country},ignore_index=True)

print("Time spent: %.0f minutes" % ((time.time() - start)/60))
worldtrends_df

In [ ]:
#Saving dataframe as .csv file
from datetime import datetime

#In case you are using google colab
from google.colab import drive
drive.mount('/content/drive') 

diretory = '/content/drive/My Drive/Colab Notebooks/DataScience/0x/'
datetime_str=str(datetime.now()).replace(':','-').replace('.','-')
archive_name=('WorldTrends_'+datetime_str+'.csv')
worldtrends_df.to_csv(diretory+archive_name,index=False)

In [ ]:
#Reading dataframe from .csv file
import pandas as pd
#In case you are using google colab
from google.colab import drive
drive.mount('/content/drive') 

diretory = '/content/drive/My Drive/Colab Notebooks/DataScience/01/'
archive_name='WorldTrends_2020-06-27 06-56-51-185617.csv'
worldtrends_df = pd.read_csv(diretory+archive_name) 

worldtrends_df

In [ ]:
#Pre-processing to remove punctuation and capital letters

import re, string

exclude=string.punctuation
regex = re.compile('[%s]' % re.escape(string.punctuation))

for index, row in worldtrends_df.iterrows():
  worldtrends_df.loc[index,'name_formated']=regex.sub('',worldtrends_df.loc[index,'name']).lower().replace(' ','')

worldtrends_df

In [ ]:
#Grouping all popular trends 
popularTrends_df=pd.DataFrame(columns=['name','volume','ncountries','countries'])

for index, row in worldtrends_df[worldtrends_df.placeTypeName=='Country'].iterrows():
  if popularTrends_df.loc[popularTrends_df.name==row['name_formated']].empty :
    popularTrends_df=popularTrends_df.append({'name':row['name_formated'],'volume':row['volume'],'ncountries':1,'countries':row['place']},ignore_index=True)
  else:
    trendrow=popularTrends_df.name==row['name_formated']
    popularTrends_df.loc[trendrow,'volume']=popularTrends_df.loc[trendrow]['volume']+row['volume']
    popularTrends_df.loc[trendrow,'ncountries']=popularTrends_df.loc[trendrow]['ncountries']+1
    popularTrends_df.loc[trendrow,'countries']=popularTrends_df.loc[trendrow]['countries'].values[0]+','+row['place']

popularTrends_df=popularTrends_df.sort_values(by=['volume'],ascending=False)
popularTrends_df

In [ ]:
#Getting font rocko (optional)
#import matplotlib.font_manager as fm

#!wget 'https://www.1001freefonts.com/d/3850/rocko.zip'
#!unzip 'rocko.zip'
#!mv "RockoFLF.ttf" "/content/drive/My Drive/Colab Notebooks/DataScience/Fonts/"

In [ ]:
#Most popular trends (Horizontal Bar chart)
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fontprop = fm.FontProperties(fname='/content/drive/My Drive/Colab Notebooks/DataScience/Fonts/RockoFLF.ttf')

trends=10
names=popularTrends_df['name'][0:trends][::-1]
volume=popularTrends_df['volume'][0:trends][::-1]
ncountries=popularTrends_df['ncountries'][0:trends][::-1]

newnames=[]
for i in range(10):
  newnames.append(names.values[i]+' ('+str(int(ncountries.values[i]))+')')

fig, ax = plt.subplots()
ax.barh(newnames, volume)
ax.set_title('Worldwide trendtopics',fontproperties=fontprop,fontsize=16)
ax.set_ylabel('Name (Countries)',fontproperties=fontprop,fontsize=12)
ax.set_xlabel('Number of Tweets',fontproperties=fontprop,fontsize=12)

#Changing xlabel
locs = ax.get_xticks()
ref=['','mil','M','B','T']
power=0
while max(locs)>1000:
  locs=locs/1000
  power+=1

xlabel=[]
for n in locs:
  xlabel.append(str(int(n))+ref[power])

ax.set_xticklabels(xlabel,fontproperties=fontprop,fontsize=12)
ax.set_yticklabels(newnames,fontproperties=fontprop,fontsize=12)

for i, v in enumerate(volume):
  ticklabel=str(int(v/(1000**power)))+ref[power]
  if i==9:
    ax.text(v-2e7, i-0.15, ticklabel,color='white',fontproperties=fontprop,fontweight='bold') 
  else:
    ax.text(v+3e6, i-0.15, ticklabel,fontproperties=fontprop,fontweight='bold')

diretory = '/content/drive/My Drive/Colab Notebooks/DataScience/0x'
plt.tight_layout()
plt.savefig(f"{diretory}/Worldwide trendtopics.png")

In [ ]:
#Most popular trends (Pie chart)
from matplotlib import cm
fontprop = fm.FontProperties(fname='/content/drive/My Drive/Colab Notebooks/DataScience/Fonts/RockoFLF.ttf')

trends=10
total=popularTrends_df['volume'].sum()
names=popularTrends_df['name'][0:trends].tolist()
volume=popularTrends_df['volume'][0:trends].tolist()

names.append('others')
volume.append(total-popularTrends_df['volume'][0:trends].sum())

newnames=[]
for i in range(trends+1):
  newnames.append(names[i]+' ('+str(int(volume[i]/total*100))+'%)')

fig, ax = plt.subplots()
cmap=plt.get_cmap("tab20")

wedges, autotexts=ax.pie(volume,colors=cmap(range(trends+1)))
ax.set_xticklabels(volume)
ax.set_title('Worldwide trendtopics',fontproperties=fontprop,fontsize=16)
ax.legend(wedges,newnames,title="Name",loc=[1,0.2],fontsize=8)

diretory = '/content/drive/My Drive/Colab Notebooks/DataScience/0x'
plt.tight_layout()
plt.savefig(f"{diretory}/Worldwide trendtopics pie.png")

In [ ]:
#Getting data from the top trend

idmax=pd.to_numeric(popularTrends_df['volume']).idxmax()
topname=popularTrends_df.loc[idmax,'name']

topname_df=worldtrends_df[worldtrends_df.name_formated.eq(topname)][['volume','place','placeTypeName']]

topname_df

In [ ]:
#Getting latitude and longitude for places
from geopy.geocoders import Nominatim

topname_df['latitude']=0.
topname_df['longitude']=0.

geolocator = Nominatim(user_agent='myapplication')
for index, row in topname_df.iterrows():
  topname_df.loc[index,'latitude']=geolocator.geocode(row['place']).latitude
  topname_df.loc[index,'longitude']=geolocator.geocode(row['place']).longitude

In [ ]:
#Heatmap - Towns
import folium
from folium import plugins
import numpy as np

topname_towndf=topname_df[topname_df['placeTypeName']=='Town']

town_latitudes=topname_towndf['latitude'].tolist()
town_longitudes=topname_towndf['longitude'].tolist()
town_volume=topname_towndf['volume'].tolist()

for i,v in enumerate(town_volume):
  town_volume[i]=v/max(town_volume)

coordinates = np.column_stack((town_latitudes, town_longitudes))
coordinates = np.column_stack((coordinates, town_volume))

map = folium.Map(zoom_start=3.)
map.add_child(plugins.HeatMap(coordinates))
map

diretory = '/content/drive/My Drive/Colab Notebooks/DataScience/0x'
map.save(f"{diretory}/HeatMap_Town.html")

In [ ]:
#Heatmap - Countries
import folium
from folium import plugins

topname_countrydf=topname_df[topname_df['placeTypeName']=='Country']

country_latitudes=topname_countrydf['latitude'].tolist()
country_longitudes=topname_countrydf['longitude'].tolist()
country_volume=topname_countrydf['volume'].tolist()
for i,v in enumerate(country_volume):
  country_volume[i]=v/max(country_volume)

coordinates = np.column_stack((country_latitudes, country_longitudes))
coordinates = np.column_stack((coordinates, country_volume))

map = folium.Map(zoom_start=3.)
map.add_child(plugins.HeatMap(coordinates))
map

diretory = '/content/drive/My Drive/Colab Notebooks/DataScience/0x'
map.save(f"{diretory}/HeatMap_Country.html")